In [1]:
from pennylane import numpy as np
import pennylane as qml

In [2]:
def generating_fourier_state(n_qubits, m):
    """Function which, given the number of qubits and an integer m, returns the circuit and the angles that generate the state
    QFT|m> following the above template.

    Args:
        - n_qubits (int): number of qubits in the circuit.
        - m (int): basis state that we generate. For example, for 'm = 3' and 'n_qubits = 4'
        we would generate the state QFT|0011> (3 in binary is 11).

    Returns:
       - (qml.QNode): circuit used to generate the state.
       - (list[float]): angles that generate the state QFT|m>.
    """

    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def circuit(angles):
        """This is the quantum circuit that we will use."""

        # QHACK #

        # Add the template of the statement with the angles passed as an argument.
        for i in range(n_qubits):
          qml.Hadamard(wires = i)
          qml.RZ(angles[i], wires = i)

        # QHACK #

        # We apply QFT^-1 to return to the computational basis.
        # This will help us to see how well we have done.
        qml.adjoint(qml.QFT)(wires=range(n_qubits))

        # We return the probabilities of seeing each basis state.
        return qml.probs(wires=range(n_qubits))

    def error(angles):
        """This function will determine, given a set of angles, how well it approximates
        the desired state. Here it will be necessary to call the circuit to work with these results.
        """

        probs = circuit(angles)
        # QHACK #

        # The return error should be smaller when the state m is more likely to be obtained.
        return np.sum(probs**2) + 1 - 2*probs[m]

        # QHACK #

    # This subroutine will find the angles that minimize the error function.
    # Do not modify anything from here.

    opt = qml.AdamOptimizer(stepsize=0.8)
    epochs = 1000

    angles = np.zeros(n_qubits, requires_grad=True)

    for epoch in range(epochs):
        angles = opt.step(error, angles)
        angles = np.clip(opt.step(error, angles), -2 * np.pi, 2 * np.pi)
        print(f"Epoch {epoch}: error: {error(angles)} angles: {angles}")

    return circuit, angles

In [3]:
inputs = [2, 3]
n_qubits = int(inputs[0])
m = int(inputs[1])

output = generating_fourier_state(n_qubits, m)
output[0](output[1])
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def check_with_arbitrary_state():
    for i in range(n_qubits):
        qml.RY(i, wires=i)
    for op in output[0].qtape.operations:
        qml.apply(op)
    return qml.state()

print(",".join([f"{p.real.round(5)},{p.imag.round(5)}" for p in check_with_arbitrary_state()]))

Epoch 0: error: 1.9998996312648831 angles: [ 1.15684360e-02 -2.60392227e-12]
Epoch 1: error: 1.0254469239114876 angles: [ 0.99147541 -0.96904087]
Epoch 2: error: 0.08553223748704708 angles: [ 2.22084705 -2.05820968]
Epoch 3: error: 0.0971882431842741 angles: [ 3.41532549 -2.534117  ]
Epoch 4: error: 0.19741724094965907 angles: [ 4.28646973 -2.22014063]
Epoch 5: error: 0.2537921584819425 angles: [ 4.53396975 -1.59128839]
Epoch 6: error: 0.14992160980062996 angles: [ 4.24264267 -1.06647797]
Epoch 7: error: 0.04445804797596442 angles: [ 3.70666999 -0.90793799]
Epoch 8: error: 0.009588747239237527 angles: [ 3.17302154 -1.0386705 ]
Epoch 9: error: 0.005487726235450463 angles: [ 2.72611792 -1.27238201]
Epoch 10: error: 0.0277463261989237 angles: [ 2.38782565 -1.51270399]
Epoch 11: error: 0.059673698647184636 angles: [ 2.22542595 -1.72307479]
Epoch 12: error: 0.0512166880830236 angles: [ 2.28568492 -1.84175317]
Epoch 13: error: 0.01824623506916967 angles: [ 2.50862278 -1.84474601]
Epoch 14: e